In [62]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/reciprocal_best_hits/01_get_best_hits")

library(pafr)
library(dplyr, verbose = FALSE)
library(stringr)
library(data.table)

options(scipen = 100, digits = 10)
myo_meta <- read.csv("../../../data/myotis_meta.csv")

In [65]:
for (i in seq_along(myo_meta$Abbr)) {
  
  for (j in seq_along(myo_meta$Abbr)) {

    if (i < j) {

      print(paste(myo_meta$Abbr[i], "vs.", myo_meta$Abbr[j]))

      working_paf1 <- read_paf(file.path("resources/pafs", intersect(
        list.files(path = "resources/pafs", pattern = paste0("^", myo_meta$Abbr[j])),
        list.files(path = "resources/pafs", pattern = paste0("_", myo_meta$Abbr[i]))
      )))

      working_paf2 <- read_paf(file.path("resources/pafs", intersect(
        list.files(path = "resources/pafs", pattern = paste0("^", myo_meta$Abbr[i])),
        list.files(path = "resources/pafs", pattern = paste0("_", myo_meta$Abbr[j]))
      )))

      spec1_intersect <- fread(file.path("resources/beds", paste0(
        myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], "_positions_intersect.bed"
      )), data.table = FALSE)

      spec2_intersect <- fread(file.path("resources/beds", paste0(
        myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], "_positions_intersect.bed"
      )), data.table = FALSE)

      working_paf1$qname <- gsub("SCAF", "SUPER", working_paf1$qname)
      working_paf2$qname <- gsub("SCAF", "SUPER", working_paf2$qname)
      working_paf1$tname <- gsub("SCAF", "SUPER", working_paf1$tname)
      working_paf2$tname <- gsub("SCAF", "SUPER", working_paf2$tname)
      spec1_intersect$V1 <- gsub("SCAF", "SUPER", spec1_intersect$V1)
      spec1_intersect$V8 <- gsub("SCAF", "SUPER", spec1_intersect$V8)
      spec2_intersect$V1 <- gsub("SCAF", "SUPER", spec2_intersect$V1)
      spec2_intersect$V8 <- gsub("SCAF", "SUPER", spec2_intersect$V8)

      working_paf1$Id <- with(working_paf1, paste(qstart, qend, tstart, tend, id, sep = "_"))      
      working_paf2$Id <- with(working_paf2, paste(qstart, qend, tstart, tend, id, sep = "_"))   

      ## Insert new sequences into original PAF files:

      target_inds <- match(working_paf1$Id, spec1_intersect$V8)
      working_paf1$tstart <- spec1_intersect$V2[target_inds]
      working_paf1$tend <- spec1_intersect$V3[target_inds]
      query_inds <- match(working_paf1$Id, spec2_intersect$V17)
      working_paf1$qstart <- spec2_intersect$V2[query_inds]
      working_paf1$qend <- spec2_intersect$V3[query_inds]

      target_inds <- match(working_paf2$Id, spec2_intersect$V8)
      working_paf2$tstart <- spec2_intersect$V2[target_inds]
      working_paf2$tend <- spec2_intersect$V3[target_inds]
      query_inds <- match(working_paf2$Id, spec1_intersect$V17)
      working_paf2$qstart <- spec1_intersect$V2[query_inds]
      working_paf2$qend <- spec1_intersect$V3[query_inds]

      working_paf1 <- working_paf1[!is.na(working_paf1$tstart),]
      working_paf2 <- working_paf2[!is.na(working_paf2$tstart),]

      ## Subset to hits found in both directions:

      working_paf1 <- working_paf1[with(working_paf1, paste(qstart, qend, tstart, tend)) %in% with(working_paf2, paste(tstart, tend, qstart, qend)),]
      working_paf2 <- working_paf2[with(working_paf2, paste(qstart, qend, tstart, tend)) %in% with(working_paf1, paste(tstart, tend, qstart, qend)),]
      working_paf2 <- working_paf2[match(with(working_paf1, paste(tstart, tend, qstart, qend)), with(working_paf2, paste(qstart, qend, tstart, tend))),]
      working_paf1 <- working_paf1[match(with(working_paf2, paste(tstart, tend, qstart, qend)), with(working_paf1, paste(qstart, qend, tstart, tend))),]

      ## Save results:
      
      working_paf1$qname <- as.character(working_paf1$qname)
      working_paf2$qname <- as.character(working_paf2$qname)
      working_paf1$qname <- gsub("#", ".", working_paf1$qname, fixed = TRUE)
      working_paf2$qname <- gsub("#", ".", working_paf2$qname, fixed = TRUE)

      write.csv(working_paf1, file = paste0(
        "results/", myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], "_best_hits.csv"
      ), row.names = FALSE, quote = FALSE)

      write.csv(working_paf2, file = paste0(
        "results/", myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], "_best_hits.csv"
      ), row.names = FALSE, quote = FALSE)

    }
  }
}

[1] "mMyoAui vs. mMyoCai"
[1] "mMyoAui vs. mMyoEvo"
[1] "mMyoAui vs. mMyoLuc"
[1] "mMyoAui vs. mMyoOcc"
[1] "mMyoAui vs. mMyoThy"
[1] "mMyoAui vs. mMyoVel"
[1] "mMyoAui vs. mMyoVol"
[1] "mMyoAui vs. mMyoYum"
[1] "mMyoCai vs. mMyoEvo"
[1] "mMyoCai vs. mMyoLuc"
[1] "mMyoCai vs. mMyoOcc"
[1] "mMyoCai vs. mMyoThy"
[1] "mMyoCai vs. mMyoVel"
[1] "mMyoCai vs. mMyoVol"
[1] "mMyoCai vs. mMyoYum"
[1] "mMyoEvo vs. mMyoLuc"
[1] "mMyoEvo vs. mMyoOcc"
[1] "mMyoEvo vs. mMyoThy"
[1] "mMyoEvo vs. mMyoVel"
[1] "mMyoEvo vs. mMyoVol"
[1] "mMyoEvo vs. mMyoYum"
[1] "mMyoLuc vs. mMyoOcc"
[1] "mMyoLuc vs. mMyoThy"
[1] "mMyoLuc vs. mMyoVel"
[1] "mMyoLuc vs. mMyoVol"
[1] "mMyoLuc vs. mMyoYum"
[1] "mMyoOcc vs. mMyoThy"
[1] "mMyoOcc vs. mMyoVel"
[1] "mMyoOcc vs. mMyoVol"
[1] "mMyoOcc vs. mMyoYum"
[1] "mMyoThy vs. mMyoVel"
[1] "mMyoThy vs. mMyoVol"
[1] "mMyoThy vs. mMyoYum"
[1] "mMyoVel vs. mMyoVol"
[1] "mMyoVel vs. mMyoYum"
[1] "mMyoVol vs. mMyoYum"
